In [2]:
from NN_Thesis.nn_classes import *
from NN_Thesis.trainer import *
from NN_Thesis.adapters import *
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
from torch import optim


from torchvision.transforms import transforms
import numpy as np
import os
from PIL import Image
import random

from matplotlib import pyplot as plt



In [3]:
from NN_Thesis.models import *

In [26]:
torch.manual_seed(123)
random.seed(123)
batch_size = 16


from NN_Thesis.dataset import cifar_n_dataset
from torch.utils.data import DataLoader

train_path = 'data/cifar_5/cifar_04/train/data'
test_path ='data/cifar_5/cifar_04/test/data'

train_data = cifar_n_dataset(train_path)
test_data = cifar_n_dataset(train_path)
print(train_data.data.shape)

def normalize_channels(data):
    #We have a nxCxWxH array
    data = torch.from_numpy(data)
    d = torch.flatten(data,2,-1).to(dtype = torch.float32)/255
    mean= torch.mean(d,dim = [0,2])
    std = torch.std(d,dim = [0,2])
    return mean,std

mean,std = normalize_channels(train_data.data)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])


test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_data.transform = train_transform
test_data.transform = test_transform



(25000, 3, 32, 32)
('airplane', 'automobile', 'bird', 'cat', 'deer')


# Code For Initial Cifar5 Training

In [3]:
train_dataloader = DataLoader(train_data,batch_size =64,shuffle = True)
test_dataloader = DataLoader(test_data,batch_size =64)

classes = tuple(train_data.label_names)

BNN_resnet18 = resnet_binary(num_classes = 5 , depth = 18, dataset = 'cifar10')
resnet18 = resnet(num_classes = 5 , depth = 18, dataset = 'cifar10')


BNN_trainer = Trainer(BNN_resnet18,name = 'BinaryCifar5',classes = classes,seed = 123,binarise = True)
resnet_trainer =Trainer(resnet18,name = 'Cifar5',classes = classes,seed = 123)

#Set Training Params
for trainer in [BNN_trainer,resnet_trainer]:
    trainer.lr = 0.1
    trainer.epochs = 100
    trainer.train(train_dataloader,test_dataloader)


# Code For Finetuning

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('Binary_ResNet_best_acc.pth',map_location= device)

BNN_adapter_model = resnet18_adapt()
BNN_adapter_model.load_state_dict(Bin_state_dict)
BNN_adapter_model.freeze()

adapter = bottleneck_adapter( (80,32,32),4)
adapter2 = bottleneck_adapter( (320,8,8),6)

# BNN_adapter_model.add_adapter(after = ['layer1','layer3'],adapter= [adapter,adapter2] )
BNN_adapter_model.add_adapter(after = 'layer1',adapter= adapter )
BNN_adapter_model.add_adapter(after = 'layer3',adapter= adapter2 )



3

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('Binary_ResNet_best_acc.pth',map_location= device)

x = torch.ones(1,3,32,32)
# k = torch.sign(torch.zeros(1,80,32,32))
BNN_adapter_model(x)


# def num_params(module):
#     return sum([p.numel() for p in module.parameters()])



# n_param = num_params(adapter)
# n_total_param = num_params(BNN_adapter_model) 
# n_per = (n_total_param-n_param)/n_total_param


# n_total_param,n_total_param-n_param, n_param,n_per*100

    

NameError: name 'BNN_adapter_model' is not defined

In [29]:
no_adapt = resnet_binary(num_classes = 10 , depth = 18, dataset = 'cifar10')
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
trainer = Trainer(no_adapt,name = 'test',classes = classes,seed = 123,binarise = True)
# resnet_trainer =Trainer(resnet18,classes = classes,seed = 123)

#Set Training Params
for trainer in [trainer]:
    trainer.lr = 0.1
    trainer.batch_size = 128
    trainer.epochs = 200
    trainer.epoch_chkpts = [150,175]
    
# trainer.train(trainloader,testloader)
# trainer.load('Binary_ResNet_latest.pth',map_location= 'cpu')

cuda:0


In [34]:
trainer.load('test_150.pth',map_location= device,load_model= True)
trainer.model.to(device)
# print(trainer.model.device)
trainer.T_max = 200
trainer.scheduler.T_max= trainer.T_max
print(trainer.scheduler.state_dict())
trainer.train(trainloader,testloader)

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'lr_scheduler_state_dict', 'loss'])
Successfully loaded model_state_dict\ ! 
{'T_max': 200, 'eta_min': 0, 'base_lrs': [0.1], 'last_epoch': 151, '_step_count': 152, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [0.0]}
start
start_epoch : 0
lr : 0.1
batch_size : 16
epochs : 1
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.1
    lr: 0.0
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
seed : 123
epoch: 151 average loss: 0.524 Epoch Time 0.02 hours
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\test_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\test_latest.pth saved!
Overall Accuracy : 82.3%
best_acc.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\test_chkpt_150.pth saved!
epoch: 152 average loss: 0.519 Epoch Time 0.03 hours
c:

In [15]:
# from NN_Thesis.models.binarized_modules import BinarizeConv2d
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('test_best_acc.pth',map_location= device)

bt_model = resnet18_adapt()
bt_model.load_state_dict(Bin_state_dict)

conv_model = resnet18_adapt()
conv_model.load_state_dict(Bin_state_dict)

conv_bt_model = resnet18_adapt()
conv_bt_model.load_state_dict(Bin_state_dict)



adapters = [
    (bottleneck_adapter((80,32,32),5),bottleneck_adapter((160,16,16),5),bottleneck_adapter((320,8,8),5)),
    (conv_adapter(80,kernel= 3,padding = 1),conv_adapter(160,kernel= 3,padding = 1),conv_adapter(320,kernel=1)),
    (conv_bottleneck_adapter(80,80,kernel=3,padding=1),conv_bottleneck_adapter(160,80,kernel=3,padding=1),conv_bottleneck_adapter(320,160,kernel=3,padding=1))
]


#Set all layer requires grad to false and set to eval mode()
for m,adpts in zip([bt_model,conv_model,conv_bt_model],adapters):
    m.freeze()
    m.add_adapter(after = 'layer1',adapter =adpts[0])
    m.add_adapter(after = 'layer2',adapter =adpts[1])
    m.add_adapter(after = 'layer3',adapter =adpts[2])


# bottle_trainer = adapter_Trainer(model = bt_model,seed = 123,name = 'Bottleneck_BNN2',classes = classes,binarise= True)
conv_trainer = adapter_Trainer(model = conv_model,seed = 123,name = 'Conv_BNN2',classes = classes,binarise= True)
conv_bt_trainer = adapter_Trainer(model = conv_bt_model,seed = 123,name = 'ConvBN_BNN2',classes = classes,binarise= True)
# trainer.load('test_150.pth',map_location= device,load_model= True)
for trainer in [conv_trainer,conv_bt_trainer]:
    m = trainer.model
    m.to(trainer.device)
    for layer in [m.fc,m.bn3,m.bn3]:
        for p in layer.parameters():
            p.requires_grad = True
        layer.train()
    trainer.lr = 0.01
    trainer.batch_size = 128
    trainer.epochs =80
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.T_max = 80
    # trainer.scheduler.T_max= trainer.T_max
    trainer.optimizer = optim.Adam(trainer.model.parameters(), lr=trainer.lr)
    trainer.scheduler = optim.lr_scheduler.ExponentialLR(trainer.optimizer, gamma=0.95)
    trainer.train(trainloader,testloader)

start
start_epoch : 0
lr : 0.01
batch_size : 128
epochs : 80
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.01
    lr: 0.01
    maximize: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 2.592 Epoch Time 0.89 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_latest.pth saved!
Overall Accuracy : 29.9%
best_acc.pth saved!
epoch: 2 average loss: 1.996 Epoch Time 1.77 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_latest.pth saved!
Overall Accuracy : 26.5%
epoch: 3 average loss: 1.882 Epoch Time 2.64 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_best_loss.pth saved!
c:\Users\John Su\Downloads\S

# DEBUGGING TIME


In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('test_best_acc.pth',map_location= device)

debug = resnet18_adapt()
debug.load_state_dict(Bin_state_dict)
debug.freeze()
debug.add_adapter(after = 'layer3',adapter = identity_adapter())

debug_trainer = adapter_Trainer(model = debug,seed = 123,name = 'debug',classes = classes,binarise= True)
for trainer in [debug_trainer]:
    m = trainer.model
    m.to(trainer.device)
    for layer in [m.fc,m.bn3,m.bn3]:
        for p in layer.parameters():
            p.requires_grad = True
        layer.train()
    trainer.lr = 0.01
    trainer.batch_size = 128
    trainer.epochs =10
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.T_max = 80
    # trainer.scheduler.T_max= trainer.T_max
    trainer.optimizer = optim.Adam(trainer.model.parameters(), lr=trainer.lr)
    trainer.scheduler = optim.lr_scheduler.ExponentialLR(trainer.optimizer, gamma=0.95)
    trainer.train(trainloader,testloader)

start
start_epoch : 0
lr : 0.01
batch_size : 128
epochs : 10
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.01
    lr: 0.01
    maximize: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 0.448 Epoch Time 0.49 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_latest.pth saved!
Overall Accuracy : 83.7%
best_acc.pth saved!
epoch: 2 average loss: 0.447 Epoch Time 0.98 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_latest.pth saved!
Overall Accuracy : 83.7%
best_acc.pth saved!
epoch: 3 average loss: 0.445 Epoch Time 1.47 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_best_loss.pth saved!
c:\Users\John Su\Downloads\S

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('test_best_acc.pth',map_location= device)

adapter = mini_bottleneck_adapter()

mini_adapt_model = resnet18_adapt()
mini_adapt_model.load_state_dict(Bin_state_dict)
mini_adapt_model.freeze()
mini_adapt_model.add_adapter(after = 'layer3',adapter = adapter)

mini_adapt_trainer = adapter_Trainer(model = mini_adapt_model,seed = 123,name = 'mini_adapt',classes = classes,binarise= True)
for trainer in [mini_adapt_trainer]:
    m = trainer.model
    m.to(trainer.device)
    for layer in [m.fc,m.bn3,m.bn3]:
        for p in layer.parameters():
            p.requires_grad = True
        layer.train()
    trainer.lr = 0.01
    trainer.batch_size = 128
    trainer.epochs =10
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.T_max = 80
    # trainer.scheduler.T_max= trainer.T_max
    trainer.optimizer = optim.Adam(trainer.model.parameters(), lr=trainer.lr)
    trainer.scheduler = optim.lr_scheduler.ExponentialLR(trainer.optimizer, gamma=0.95)
    trainer.train(trainloader,testloader)

start
start_epoch : 0
lr : 0.01
batch_size : 128
epochs : 10
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.01
    lr: 0.01
    maximize: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 0.447 Epoch Time 0.56 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_latest.pth saved!
Overall Accuracy : 83.6%
best_acc.pth saved!
epoch: 2 average loss: 0.447 Epoch Time 1.12 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_latest.pth saved!
Overall Accuracy : 83.7%
best_acc.pth saved!
epoch: 3 average loss: 0.444 Epoch Time 1.69 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_best_loss.pth saved!
c:\